In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sqlalchemy as s
kengine = s.create_engine('sqlite:///khipu.db')
cengine = s.create_engine('sqlite:///collca.db')


def df_from_SQL(db,query):
    engine = s.create_engine('sqlite:///khipu.db')
    if 'r' in db:
        engine = s.create_engine('sqlite:///RobynDatabase.db')
    elif 'c' in db:
        engine = s.create_engine('sqlite:///collca.db')
    df = pd.read_sql_query(query,engine)
    return df


## Find duplicate khipus

In [8]:
# for each khipu, get a list of unique knot values. 
#get a df with all khipu ids
df=df_from_SQL('c','select khipu_id as qID from khipu_main')
print(df.head())

       qID
0      996
1  1000000
2  1000001
3  1000002
4  1000003
Empty DataFrame
Columns: [qID, value]
Index: []


In [20]:
valueDF = pd.DataFrame(columns = ['qID', 'value'])

#iterate through the ids
for index,row in df.iterrows():
#in each iteration, get the unique values for each khipu
    df1=df_from_SQL('c','select distinct(cord_value) from cord where khipu_id = '+str(row['qID']))
    #print(df1.head())
    if df1.index.size>1:
        for index1,row1 in df1.iterrows():
            #save unique values to a list on the main df
            if row1['cord_value']>0:
                df2 = {'qID': row['qID'], 'value': row1['cord_value'].astype(int)}
               # print(df2)
                valueDF = valueDF.append(df2, ignore_index = True)

valueDF.head()

,qID,value
0,996,16
1,1000000,18
2,1000000,1
3,1000000,10
4,1000000,28


In [48]:
df['match0']=''
df['perc0']=''
df['match1']=''
df['perc1']=''
df['match2']=''
df['perc2']=''
    

#iterate through the khipus again
for index,row in df.iterrows():
    #for each khipu, pull up the list
    #print(row['qID'])
    uniqueVal = valueDF[valueDF['qID']==row['qID']]['value'].unique()
    #go through each list of unique values and get a count of how many match
    if uniqueVal.size >4:
        tempDF = df.copy()
        tempDF['perc']=''
        tempDF = tempDF.drop([index]) #to remove the duplicate entry
        for index1,row1 in tempDF.iterrows():
            uniqueVal2 = valueDF[valueDF['qID']==row1['qID']]['value'].unique()
            #print(str(row['qID'])+': '+str(uniqueVal2))
            #go through each list of unique values and get a count of how many match
            matches=0
            if uniqueVal2.size>4:
                #print(str(row['qID'])+' size: '+str(uniqueVal2.size))
            
                for val in uniqueVal:
                    if val in uniqueVal2:
                        matches = matches+1
                #get a percent match for each khipu: (number of matches)/(list1+list2 number of distinct values in both lists)
                perc = matches/(uniqueVal.size+uniqueVal2.size-matches)
                tempDF.at[index1,'perc']=perc
                #print(perc)
            else:
                #print('drop index '+str(index1))
                tempDF = tempDF.drop([index1])
        #print(tempDF.head())
        tempDF = tempDF.dropna(subset=['perc'])
        tempDF = tempDF.sort_values(by=['perc'],ascending=False)
        #print(tempDF.head())
        
        #print the top 3 and their percents
        #print(tempDF.iloc[0,0])
        #print(tempDF.iloc[0,7])
        df.at[index,'match0']=tempDF.iloc[0,0]
        df.at[index,'match1']=tempDF.iloc[1,0]
        df.at[index,'match2']=tempDF.iloc[2,0]
        df.at[index,'perc0']=tempDF.iloc[0,7]
        df.at[index,'perc1']=tempDF.iloc[1,7]
        df.at[index,'perc2']=tempDF.iloc[2,7]
        
    else:
        df = df.drop([index])
print(df.head())

       qID   match0     perc0   match1     perc1   match2     perc2
1  1000000  1000567  0.545455  1000162       0.5  1000287       0.5
2  1000001  1000319  0.230769  1000156  0.222222  1000133  0.222222
3  1000002  1000200      0.15  1000621     0.125  1000291     0.125
4  1000003  1000316  0.297872  1000021  0.265306  1000385  0.255319
5  1000004  1000385  0.574468  1000514  0.551724  1000140  0.535714


In [51]:
df = df.sort_values(['perc0'],ascending = False)
df.head()

,qID,match0,perc0,match1,perc1,match2,perc2
169,1000169,1000006,1.0,1000569,0.833333,1000656,0.714286
334,1000346,1000230,1.0,1000020,0.909091,1000098,0.909091
299,1000307,1000068,1.0,1000305,0.833333,1000143,0.833333
223,1000227,1000135,1.0,1000646,1.0,1000220,0.75
7,1000006,1000169,1.0,1000569,0.833333,1000103,0.714286
